# Upgrade NLTK

In [ ]:
!pip install --upgrade nltk

     |████████████████████████████████| 1.4MB 6.0MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434674 sha256=445abf259fcbf33b3b97a9a068a1b48676bafe5c36c5baac2f7aea9537617621
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


# Load Libraries

In [ ]:
from numpy import argmax
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
from tqdm import tqdm
import numpy as np
import pickle
import tensorflow as tf
import keras
from nltk.translate import meteor_score
import pickle
import nltk
nltk.download('wordnet')

ImportError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def save_obj(obj, name ):
    with open('/content/drive/MyDrive/flickr_30k/RG_trial_models/obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('/content/drive/MyDrive/flickr_30k/RG_trial_models/obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
test_descriptions = load_obj('test_descriptions_11282020')
test_features = load_obj('test_features_11282020')

with open('/content/drive/MyDrive/flickr_30k/RG_trial_models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

max_len = 70

In [ ]:
# previously, the best model has been determined to be 128, epoch 12, beam size 2
def word_for_id(integer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
  
def generate_desc_beam(model, photo, beam_size):
  in_text_beam = [[tokenizer.texts_to_sequences(['startseq'])[0],0]]
	# iterate over the whole length of the sequence
  while len(in_text_beam[0][0]) < max_len:
    # for every set of predictions
    temp = []
    for i in in_text_beam:
      sequence = i[0]
      sequence = pad_sequences([sequence], maxlen=max_len, padding='post')
      prediction = model.predict([photo, sequence], verbose=0)
      top_predictions = np.argsort(prediction[0])[-beam_size:]
      for j in top_predictions:
        next_seq = i[0][:] + [j]
        next_prob = i[1] + prediction[0][j]
        temp += [[next_seq, next_prob]]
    in_text_beam = temp
    in_text_beam = sorted(in_text_beam, reverse=False, key=lambda x: x[1])
    in_text_beam = in_text_beam[-beam_size:]
  in_text_beam = in_text_beam[-1][0]
  final_text = 'startseq'
  intermediate_caption = [word_for_id(i) for i in in_text_beam]
  for word in intermediate_caption[1:]:
    final_text += ' ' + word
    if word == 'endseq':
      break
  return final_text

In [ ]:
def evaluate_model(model, descriptions, photos):
  actual, predicted_beam,  = list(), list()
# step over the whole set
  for key, desc_list in tqdm(descriptions.items()):
    references = [d.split() for d in desc_list]
    actual.append(references)
    # generate description
    yhat2 = generate_desc_beam(model, photos[key], 2)
    predicted_beam.append(yhat2.split())

  return actual,  predicted_beam

In [ ]:
model_loc2 = '/content/drive/MyDrive/flickr_30k/RG_trial_models/model_126_112820_12.h5'
curr_model  = tf.keras.models.load_model(model_loc2)
actual_test, predicted_test = evaluate_model(curr_model, test_descriptions, test_features)
save_obj(actual_test, 'actual_test_captions_12012020')
save_obj(predicted_test, 'predicted_test_captions_12012020')

100%|██████████| 6357/6357 [8:04:56<00:00,  4.58s/it]


In [ ]:
# look at football image specifically for use in report
model_loc2 = '/content/drive/MyDrive/flickr_30k/RG_trial_models/model_126_112820_12.h5'
curr_model  = tf.keras.models.load_model(model_loc2)
football = load_obj('footballimage')
generate_desc_beam(curr_model, football,2)

'startseq two men in white uniforms one in white and white uniform are trying to get the ball endseq'

In [ ]:
actual_test = load_obj('actual_test_captions_12012020')
predicted_test = load_obj('predicted_test_captions_12012020')

In [ ]:
BLEU_results_predict = []
METEOR_results_predict = []

BLEU_results_actual = []
METEOR_results_actual = []

for picture in predicted_test:
  curr_pic_list_bleu = []
  curr_pic_list_met = []
  # remove startseq and endseq tags
  desc_bleu = picture[1:-1]
  desc_met = ' '.join(picture[1:-1])
  BLEU_results_predict += [desc_bleu]
  METEOR_results_predict += [desc_met]

for picture in actual_test:
  curr_pic_list_bleu = []
  curr_pic_list_met = []
  for desc in picture:
    # remove startseq and endseq tags
    desc_bleu = desc[1:-1]
    curr_pic_list_bleu +=[desc_bleu]
    desc_met = ' '.join(desc[1:-1])
    curr_pic_list_met +=[desc_met]
  BLEU_results_actual += [curr_pic_list_bleu]
  METEOR_results_actual += [curr_pic_list_met]

# Evaluate performance of final model on Test set


In [ ]:
count = 0
beam_total2 = 0

for i in range(len(METEOR_results_actual)):
  actual = METEOR_results_actual[i]
  predicted = METEOR_results_predict[i]
  curr_val_beam2 = meteor_score.meteor_score(actual, predicted)

  count += 1
  beam_total2 += curr_val_beam2

print('METEOR SCORE, beam2:', beam_total2/count)

print('BLEU-1: %f' % corpus_bleu(BLEU_results_actual, BLEU_results_predict, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu(BLEU_results_actual, BLEU_results_predict, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu(BLEU_results_actual, BLEU_results_predict, weights=(0.33, 0.33, 0.33, 0)))
print('BLEU-4: %f' % corpus_bleu(BLEU_results_actual, BLEU_results_predict, weights=(0.25, 0.25, 0.25, 0.25)))

METEOR SCORE, beam2: 0.27801075310186874
BLEU-1: 0.487420
BLEU-2: 0.288106
BLEU-3: 0.172059
BLEU-4: 0.096829
